# Dependancies

In [48]:
import pandas as pd
import pickle
import os
import numpy as np
#import tensorflow as tf
#import ltc_model as ltc
#from ctrnn_model import CTRNN, NODE, CTGRU
import argparse
from itertools import zip_longest

# Variable Control Sliders

In [9]:
data_directory = r'G:\code\Y2S1\CS3244\Data\col\collated'
counter = 0
lowest_nest = 258

# POSE DATA HANDLING


## Pandas Pickle Extract 

In [67]:
def np_list_pad(list):
    def find_shape(seq):
        try:
            len_ = len(seq)
        except TypeError:
            return ()
        shapes = [find_shape(subseq) for subseq in seq]
        return (len_,) + tuple(max(sizes) for sizes in zip_longest(*shapes,
                                                                    fillvalue=1))

    def fill_array(arr, seq):
        if arr.ndim == 1:
            try:
                len_ = len(seq)
            except TypeError:
                len_ = 0
            arr[:len_] = seq
            arr[len_:] = np.nan
        else:
            for subarr, subseq in zip_longest(arr, seq, fillvalue=()):
                fill_array(subarr, subseq)
    arr = np.empty(find_shape(list))
    fill_array(arr, list)
    return arr

class Data:
    
    def __init__(self,seq_len=16):
        data_payload = {}
        name_dict={}
        counter = 0
        global data_directory
        print(data_directory)
        train_x = []
        train_y = []
        test_x = []
        test_y = []
        for file in os.listdir(data_directory):
            if file.endswith(".pkl"):
                counter +=1 
                file_directory = os.path.join(data_directory, file)
                # print(os.path.join(data_directory, file))
                object = pd.read_pickle(file_directory)
                # print(object[1])
                sep = '_'
                stripped_file_name = file.split(sep, 1)[0]
                name_dict[counter] = stripped_file_name
                train, test = object[:int(len(object)*0.8)] , object[int(len(object)*0.8):]
                train_x.extend(train)
                test_x.extend(test)
                train_y.extend([counter]*len(train))
                test_y.extend([counter]*len(test))
                data_payload[stripped_file_name] = object
            #print(data_payload)
            # Dictonary : {Phrase: 3d array}
            # 3d array nesting is as follow: Scene,Frame, Flattened dataPoints

        train_x = np_list_pad(train_x)
        train_y = np_list_pad(train_y)
        test_x = np_list_pad(test_x)
        test_y = np_list_pad(test_y)
        print(test_x.shape)
        print(test_y)
        print(len(train_x[:2]))

        print("Total number of training sequences: {}".format(train_x.shape[0]))
        # permutation = np.random.RandomState(893429).permutation(train_x.shape[0])
        # valid_size = int(0.1*train_x.shape[0])
        # print("Validation split: {}, training split: {}".format(valid_size,train_x.shape[0]-valid_size))
        # print(permutation)
        # self.valid_x = train_x[:,permutation[:valid_size]]
        # self.valid_y = train_y[:,permutation[:valid_size]]
        # self.train_x = train_x[:,permutation[valid_size:]]
        # self.train_y = train_y[:,permutation[valid_size:]]

        # self.test_x = test_x
        # self.test_y = test_y
        # print("Total number of test sequences: {}".format(self.test_x.shape[1]))
    
    def iterate_train(self,batch_size=16):
        total_seqs = self.train_x.shape[1]
        permutation = np.random.permutation(total_seqs)
        total_batches = total_seqs // batch_size

        for i in range(total_batches):
            start = i*batch_size
            end = start + batch_size
            batch_x = self.train_x[:,permutation[start:end]]
            batch_y = self.train_y[:,permutation[start:end]]
            yield (batch_x,batch_y)
        
data = Data()

G:\code\Y2S1\CS3244\Data\col\collated
(120, 67, 258)
[ 1.  1.  2.  2.  3.  3.  3.  4.  4.  4.  5.  5.  6.  6.  6.  7.  7.  7.
  8.  8.  9.  9. 10. 10. 10. 11. 11. 12. 12. 12. 13. 13. 14. 14. 14. 14.
 15. 15. 15. 16. 16. 17. 17. 17. 18. 18. 18. 19. 19. 19. 20. 20. 20. 20.
 21. 21. 22. 22. 22. 23. 23. 24. 24. 25. 25. 25. 26. 26. 26. 26. 27. 27.
 27. 28. 28. 29. 29. 29. 30. 30. 30. 31. 31. 31. 31. 31. 31. 31. 31. 31.
 31. 31. 31. 32. 32. 32. 33. 33. 33. 34. 34. 34. 35. 35. 36. 36. 36. 36.
 37. 37. 37. 38. 38. 38. 39. 39. 40. 40. 41. 41.]
2
Total number of training sequences: 405


## Alternative Pickle Extract

In [ ]:
'''
Replace array_values line with this if you dont have pandas:

infile = open(r'G:\code\Y2S1\CS3244\Data\col\collated\afraid_scenes.pkl', 'rb')
array_values = pickle.load(infile)
infile.close()
# print(len(array_values))
'''

# Model


In [ ]:


class Model:

    def __init__(self,model_type,model_size,learning_rate = 0.001):
        self.model_type = model_type
        self.constrain_op = None
        self.x = tf.placeholder(dtype=tf.float32,shape=[None,None,lowest_nest])
        self.target_y = tf.placeholder(dtype=tf.int32,shape=[None,None])

        self.model_size = model_size
        head = self.x
        if(model_type == "lstm"):
            self.fused_cell = tf.nn.rnn_cell.LSTMCell(model_size)
            head,_ = tf.nn.dynamic_rnn(self.fused_cell,head,dtype=tf.float32,time_major=True)
        elif(model_type.startswith("ltc")):
            learning_rate = 0.01 # LTC needs a higher learning rate
            self.wm = ltc.LTCCell(model_size)
            if(model_type.endswith("_rk")):
                self.wm._solver = ltc.ODESolver.RungeKutta
            elif(model_type.endswith("_ex")):
                self.wm._solver = ltc.ODESolver.Explicit
            else:
                self.wm._solver = ltc.ODESolver.SemiImplicit

            head,_ = tf.nn.dynamic_rnn(self.wm,head,dtype=tf.float32,time_major=True)
            self.constrain_op = self.wm.get_param_constrain_op()
        elif(model_type == "node"):
            self.fused_cell = NODE(model_size,cell_clip=-1)
            head,_ = tf.nn.dynamic_rnn(self.fused_cell,head,dtype=tf.float32,time_major=True)
        elif(model_type == "ctgru"):
            self.fused_cell = CTGRU(model_size,cell_clip=-1)
            head,_ = tf.nn.dynamic_rnn(self.fused_cell,head,dtype=tf.float32,time_major=True)
        elif(model_type == "ctrnn"):
            self.fused_cell = CTRNN(model_size,cell_clip=-1,global_feedback=True)
            head,_ = tf.nn.dynamic_rnn(self.fused_cell,head,dtype=tf.float32,time_major=True)
        else:
            raise ValueError("Unknown model type '{}'".format(model_type))


        self.y = tf.layers.Dense(6,activation=None)(head)
        print("logit shape: ",str(self.y.shape))
        self.loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(
            labels = self.target_y,
            logits = self.y,
        ))
        optimizer = tf.train.AdamOptimizer(learning_rate)
        self.train_step = optimizer.minimize(self.loss)

        model_prediction = tf.argmax(input=self.y, axis=2)
        self.accuracy = tf.reduce_mean(tf.cast(tf.equal(model_prediction, tf.cast(self.target_y,tf.int64)), tf.float32))

        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())

        self.result_file = os.path.join("results","har","{}_{}.csv".format(model_type,model_size))
        if(not os.path.exists("results/har")):
            os.makedirs("results/har")
        if(not os.path.isfile(self.result_file)):
            with open(self.result_file,"w") as f:
                f.write("best epoch, train loss, train accuracy, valid loss, valid accuracy, test loss, test accuracy\n")

        self.checkpoint_path = os.path.join("tf_sessions","har","{}".format(model_type))
        if(not os.path.exists("tf_sessions/har")):
            os.makedirs("tf_sessions/har")
            
        self.saver = tf.train.Saver()

    def save(self):
        self.saver.save(self.sess, self.checkpoint_path)

    def restore(self):
        self.saver.restore(self.sess, self.checkpoint_path)


    def fit(self,gesture_data,epochs,verbose=True,log_period=50):

        best_valid_accuracy = 0
        best_valid_stats = (0,0,0,0,0,0,0)
        self.save()
        for e in range(epochs):
            if(e%log_period == 0):
                test_acc,test_loss = self.sess.run([self.accuracy,self.loss],{self.x:gesture_data.test_x,self.target_y: gesture_data.test_y})
                valid_acc,valid_loss = self.sess.run([self.accuracy,self.loss],{self.x:gesture_data.valid_x,self.target_y: gesture_data.valid_y})
                # Accuracy metric -> higher is better
                if(valid_acc > best_valid_accuracy and e > 0):
                    best_valid_accuracy = valid_acc
                    best_valid_stats = (
                        e,
                        np.mean(losses),np.mean(accs)*100,
                        valid_loss,valid_acc*100,
                        test_loss,test_acc*100
                    )
                    self.save()

            losses = []
            accs = []
            for batch_x,batch_y in gesture_data.iterate_train(batch_size=16):
                acc,loss,_ = self.sess.run([self.accuracy,self.loss,self.train_step],{self.x:batch_x,self.target_y: batch_y})
                if(not self.constrain_op is None):
                    self.sess.run(self.constrain_op)

                losses.append(loss)
                accs.append(acc)

            if(verbose and e%log_period == 0):
                print("Epochs {:03d}, train loss: {:0.2f}, train accuracy: {:0.2f}%, valid loss: {:0.2f}, valid accuracy: {:0.2f}%, test loss: {:0.2f}, test accuracy: {:0.2f}%".format(
                    e,
                    np.mean(losses),np.mean(accs)*100,
                    valid_loss,valid_acc*100,
                    test_loss,test_acc*100
                ))
            if(e > 0 and (not np.isfinite(np.mean(losses)))):
                break
        self.restore()
        best_epoch,train_loss,train_acc,valid_loss,valid_acc,test_loss,test_acc = best_valid_stats
        print("Best epoch {:03d}, train loss: {:0.2f}, train accuracy: {:0.2f}%, valid loss: {:0.2f}, valid accuracy: {:0.2f}%, test loss: {:0.2f}, test accuracy: {:0.2f}%".format(
            best_epoch,
            train_loss,train_acc,
            valid_loss,valid_acc,
            test_loss,test_acc
        ))
        with open(self.result_file,"a") as f:
            f.write("{:03d}, {:0.2f}, {:0.2f}, {:0.2f}, {:0.2f}, {:0.2f}, {:0.2f}\n".format(
            best_epoch,
            train_loss,train_acc,
            valid_loss,valid_acc,
            test_loss,test_acc
        ))